In [1]:
import os
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from cv2.dnn import readNet, blobFromImage, NMSBoxes, readNetFromCaffe

# Paths
input_path = r'data'
output_path = r'resuult\data_result'
os.makedirs(output_path, exist_ok=True)

# Load YOLOv3 model and configuration
net = readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to classify based on filename
def classify_face(filename):
    return 'real' if filename.startswith('P') else 'not real'

# Function for Non-Maximum Suppression (NMS)
def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2] + boxes[:,0]
    y2 = boxes[:,3] + boxes[:,1]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

# Initialize metrics
TP = TN = FP = FN = 0
y_true_faces = []
y_pred_faces = []
y_true_objects = []
y_pred_objects = []

# Load face detection model
prototxt_path = "deploy.prototxt"  # Update with your path if needed
caffemodel_path = "res10_300x300_ssd_iter_140000_fp16.caffemodel"  # Update with your path if needed
face_net = readNetFromCaffe(prototxt_path, caffemodel_path)

# Process each image
for filename in os.listdir(input_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        img = cv2.imread(os.path.join(input_path, filename))
        height, width, channels = img.shape

        # Data Augmentation
        img_augmented = datagen.random_transform(img)
        
        # Convert to grayscale and apply CLAHE to the entire image
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        gray_image = cv2.cvtColor(img_augmented, cv2.COLOR_BGR2GRAY)
        clahe_image = clahe.apply(gray_image)
        clahe_image = cv2.GaussianBlur(clahe_image, (3, 3), 0)
        
        # Face detection with a deep learning-based model (using SSD)
        blob = blobFromImage(cv2.resize(img_augmented, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        face_net.setInput(blob)
        detections = face_net.forward()

        faces = []
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.4:  # Lowered threshold
                box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                faces.append([x, y, x1 - x, y1 - y])

        faces = non_max_suppression(faces, 0.3)
        classification = classify_face(filename)
        
        if len(faces) == 0:
            TN += (classification == 'not real')
            FN += (classification == 'real')
            y_true_faces.append(0)
            y_pred_faces.append(0)
        else:
            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), 2)
                cv2.putText(img, classification, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
                
            TP += (classification == 'real')
            FP += (classification == 'not real')
            y_true_faces.append(1)
            y_pred_faces.append(1)
        
        # Prepare image for YOLO object detection
        blob = blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        
        # Get detection results
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            detection = out.reshape(-1, 85)
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:  # Lowered threshold
                    center_x = int(obj[0] * width)
                    center_y = int(obj[1] * height)
                    w = int(obj[2] * width)
                    h = int(obj[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply Non-Max Suppression
        indexes = NMSBoxes(boxes, confidences, 0.5, 0.4)
        
        if len(indexes) > 0:
            y_pred_objects.append(1)
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                color = (0, 255, 0)
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, f"{label} {int(confidence * 100)}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        else:
            y_pred_objects.append(0)

        # Update y_true_objects for object detection
        true_label = classify_face(filename)
        if true_label == 'real':
            y_true_objects.append(1)
        else:
            y_true_objects.append(0)
        
        # Save the processed image
        cv2.imwrite(os.path.join(output_path, filename), img)
        print(f'Saved {filename} with classification {classification}')

# Print debug information
print("Length of y_true_faces:", len(y_true_faces))
print("Length of y_pred_faces:", len(y_pred_faces))
print("Length of y_true_objects:", len(y_true_objects))
print("Length of y_pred_objects:", len(y_pred_objects))

# Calculate performance metrics if lengths match
if len(y_true_objects) == len(y_pred_objects):
    total_images = TP + TN + FP + FN
    print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    print(f"Accuracy: {(TP + TN) / total_images * 100:.2f}%")

    # Confusion Matrix and Classification Report for Face Detection
    conf_matrix_faces = confusion_matrix(y_true_faces, y_pred_faces)
    print("Confusion Matrix for Faces:\n", conf_matrix_faces)
    print("Classification Report for Faces:\n", classification_report(y_true_faces, y_pred_faces))

    # Confusion Matrix and Classification Report for Object Detection
    conf_matrix_objects = confusion_matrix(y_true_objects, y_pred_objects)
    print("Confusion Matrix for Objects:\n", conf_matrix_objects)
    print("Classification Report for Objects:\n", classification_report(y_true_objects, y_pred_objects))
else:
    print("Mismatch in the length of true and predicted")


Saved N_01.jpg with classification not real
Saved N_02.jpg with classification not real
Saved N_03.jpg with classification not real
Saved N_04.jpg with classification not real
Saved N_05.jpg with classification not real
Saved N_06.jpg with classification not real
Saved N_07.jpg with classification not real
Saved N_08.jpg with classification not real
Saved N_09.jpg with classification not real
Saved N_10.jpg with classification not real
Saved N_11.jpg with classification not real
Saved N_12.jpg with classification not real
Saved N_13.jpg with classification not real
Saved N_14.jpg with classification not real
Saved N_15.jpg with classification not real
Saved N_16.jpg with classification not real
Saved N_17.jpg with classification not real
Saved N_18.jpg with classification not real
Saved N_19.jpg with classification not real
Saved N_20.jpg with classification not real
Saved N_21.jpg with classification not real
Saved N_22.jpg with classification not real
Saved N_23.jpg with classificati